In [491]:
# GENERAL SEARCH ALGORITHMS

In [492]:
DEBUG = False
DEFAULT_ALG="BFS"

In [493]:
from dataclasses import dataclass, field
from typing import Any

class Problem:
    def __init__(self, initial_state, actions, result, goal_test, step_cost):
        self.initial_state = initial_state
        self.actions = actions
        self.result = result
        self.goal_test = goal_test
        self.step_cost = step_cost
        
@dataclass(order=True)        
class Node:
    heuristic_cost: int
    path_cost: int=field(compare=False)
    state: str=field(compare=False)
    parent: str=field(compare=False, repr=False)
    action: str=field(compare=False, repr=False)

In [494]:


def child_node(problem, parent, action, heuristics, alg):
    heuristic_estimation=0
    if(alg=="UCS"):
        return Node(parent.path_cost + problem.step_cost(parent.state, action),
               parent.path_cost + problem.step_cost(parent.state, action),
               problem.result(parent.state, action),
               parent,
               action)
    elif(alg=="Astar"):
        heuristic_estimation=heuristics[parent.state]+parent.path_cost
        return Node(heuristic_estimation,
                parent.path_cost + problem.step_cost(parent.state, action),
                problem.result(parent.state, action),
                parent,
                action)

    return Node(heuristic_estimation,
                parent.path_cost + problem.step_cost(parent.state, action),
                problem.result(parent.state, action),
                parent,
                action)

In [495]:
from queue import Queue, LifoQueue, PriorityQueue

def create_frontier(problem, alg,heuristics):
    queue = None
    match alg:
        case "BFS":
            queue = Queue()
        case "DFS":
            queue = LifoQueue()
        case "UCS":
            queue = PriorityQueue()
        case "Astar":
            queue = PriorityQueue()
            h1 = heuristics[problem.initial_state]
            queue.put(Node(h1, 0, problem.initial_state, None, None))
            return queue


    queue.put(Node(0, 0, problem.initial_state, None, None))
    return queue

def select_next_node(frontier):
    return frontier.get()

def expand(leaf, problem, frontier, explored,heuristics, alg):
    for action in problem.actions(leaf.state):
        child = child_node(problem, leaf, action, heuristics, alg)
        if child.state not in explored:
            #Node.path_cost-(heuristics[parent.state])
            frontier.put(child)
        

In [496]:
def graph_search(problem, alg=DEFAULT_ALG):
    frontier = create_frontier(problem, alg,heuristics)
    explored = set()
    while True:
        debug("\n Frontier:{} \n Explored".format(list(frontier.queue), explored))
        if frontier.empty():
            raise Exception("Unable to find solution")
        leaf = select_next_node(frontier)
        if problem.goal_test(leaf.state):
            print("\n Solution:\n")
            pretty_print_solution(leaf)
            return
        explored.add(leaf.state)
        expand(leaf, problem, frontier, explored,heuristics,alg)
        

In [497]:
def pretty_print_solution(leaf):
    node = leaf
    path = []
    while node.parent:
        path.append((node.state, node.action, node.path_cost))
        node = node.parent
    print("Initial State: {}".format(node.state))
    path.reverse()
    for state, action, cost in path:
        print("Action: {}   Result: {}   Cost: {}".format(action, state, cost))
    print("Goal Achieved")
        

def debug(str):
    if DEBUG:
        print(str)

In [498]:
# TAXI AGENT PROBLEM

In [499]:
![romania_map.jpg](attachment:romania_map.jpg)

/bin/bash: -c: line 1: syntax error near unexpected token `attachment:romania_map.jpg'
/bin/bash: -c: line 1: `[romania_map.jpg](attachment:romania_map.jpg)'


In [500]:
## Romania Graph Construction & Testing

In [501]:
import pandas as pd
data=pd.read_csv("/home/adarsh/Downloads/AIKR/Assignment3/Adarsh N L - Indian_capitals - NS_Dataset_1.csv")

def createCapitalcitiesGraph(): #Function to create graph
    capitalList=[]
    cost=[]
    for i in range(len(data["City"])):
        cost=tuple((data["City"][i],data["Nearest city"][i],data["Distance"][i])) #forming a tuple of city1, city2 and the distance between two cities
        capitalList.append(cost)
    return capitalList

capitalGraph=createCapitalcitiesGraph()
graph=capitalGraph
#print(graph)

In [502]:
# function to calculate the heuristics
from geopy.distance import geodesic

dataset = pd.read_csv("/home/adarsh/Downloads/AIKR/Assignment3/in.csv")

#empty list to store the heuristic estiamtion from all cities to the goal node
heuristics=[] 

goalcity="Imphal"
for i in range(len(dataset["city"])):                      #iterates over the length of the file to find the latitude and longitude of the goal state
    if(dataset["city"][i]==goalcity):
        city2=[((dataset["lat"][i]),dataset["lng"][i])]

def distance_between_cities(city1,city2): #calculates the straight line distance between the ith city and the goal city
    dist=geodesic(city1,city2).kilometers
    return dist

for i in range(len(dataset["city"])):
    if(dataset["capital"][i]=="admin"):
        city1=[((dataset["lat"][i]),dataset["lng"][i])]
        heuristics.append((dataset["city"][i],distance_between_cities(city1,city2)))

#print(dict(heuristics))
heuristics=dict(heuristics)
#heuristics['Chennai']

In [503]:
# An utility function to create a collection of all city names
def cities_set(graph):
    cities = []
    for city1, city2, _ in graph:
        cities.extend([city1, city2])
    return sorted(set(cities))           

In [504]:
# An utility to print the list of cities and the total - sanity check
cities = cities_set(graph)
for idx, city in enumerate(cities):
    print("{}. {}".format(idx+1,city))


print("total cities: ",len(cities))


1. Agartala
2. Aizawl
3. Amaravathi
4. Bangalore
5. Bhopal
6. Bhubaneswar
7. Chandigarh
8. Chennai
9. Dehradun
10. Dispur
11. Gandhinagar
12. Gangtok
13. Hyderabad
14. Imphal
15. Itanagar
16. Jaipur
17. Kohima
18. Kolkata
19. Lucknow
20. Mumbai
21. Panaji
22. Patna
23. Raipur
24. Ranchi
25. Shillong
26. Shimla
27. Srinagar
28. Thiruvananthapuram
total cities:  28


In [505]:
graph_dict = dict()

def add_route(city1, city2, cost, graph_dict):
    if city1 not in graph_dict:
        graph_dict[city1] = {city2 : cost}
    else:
        graph_dict[city1][city2] = cost

def construct_graph_dict(graph, graph_dict):
    for city1, city2, cost in graph:
        add_route(city1, city2, cost, graph_dict)
        add_route(city2, city1, cost, graph_dict)

In [506]:
construct_graph_dict(graph, graph_dict)

In [507]:
#len(graph_dict.keys())

In [508]:
## Taxi Agent Problem Definition

In [509]:
# Initial State
ta_initial_state = "Chennai"

# Actions
def ta_actions(state):
    return list(graph_dict[state].keys())

#Transitions/Results
def ta_result(state, action):
    return action

# Goal Test
def ta_goal_test(state):
    return state == "Imphal"

# Cost Function
def ta_cost(state, action):
    return graph_dict[state][action]

taxi_problem = Problem(ta_initial_state, ta_actions, ta_result, ta_goal_test, ta_cost)

In [510]:
# The default is BFS
graph_search(taxi_problem)


 Solution:

Initial State: Chennai
Action: Amaravathi   Result: Amaravathi   Cost: 552
Action: Bhubaneswar   Result: Bhubaneswar   Cost: 1439
Action: Kolkata   Result: Kolkata   Cost: 1980
Action: Dispur   Result: Dispur   Cost: 3115
Action: Imphal   Result: Imphal   Cost: 3697
Goal Achieved


In [511]:
graph_search(taxi_problem, "DFS")


 Solution:

Initial State: Chennai
Action: Thiruvananthapuram   Result: Thiruvananthapuram   Cost: 871
Action: Bangalore   Result: Bangalore   Cost: 1701
Action: Hyderabad   Result: Hyderabad   Cost: 2370
Action: Mumbai   Result: Mumbai   Cost: 3189
Action: Raipur   Result: Raipur   Cost: 4380
Action: Ranchi   Result: Ranchi   Cost: 5060
Action: Patna   Result: Patna   Cost: 5487
Action: Kolkata   Result: Kolkata   Cost: 6170
Action: Dispur   Result: Dispur   Cost: 7305
Action: Kohima   Result: Kohima   Cost: 7755
Action: Imphal   Result: Imphal   Cost: 7991
Goal Achieved


In [512]:
graph_search(taxi_problem, "UCS")


 Solution:

Initial State: Chennai
Action: Amaravathi   Result: Amaravathi   Cost: 552
Action: Bhubaneswar   Result: Bhubaneswar   Cost: 1439
Action: Kolkata   Result: Kolkata   Cost: 1980
Action: Dispur   Result: Dispur   Cost: 3115
Action: Imphal   Result: Imphal   Cost: 3697
Goal Achieved


In [513]:
graph_search(taxi_problem, "Astar")


 Solution:

Initial State: Chennai
Action: Amaravathi   Result: Amaravathi   Cost: 552
Action: Bhubaneswar   Result: Bhubaneswar   Cost: 1439
Action: Kolkata   Result: Kolkata   Cost: 1980
Action: Dispur   Result: Dispur   Cost: 3115
Action: Imphal   Result: Imphal   Cost: 3697
Goal Achieved


In [514]:
# VACCUM CLEANER AGENT PROBLEM

In [515]:
![VacuumWorld.jpeg](attachment:VacuumWorld.jpeg)

/bin/bash: -c: line 1: syntax error near unexpected token `attachment:VacuumWorld.jpeg'
/bin/bash: -c: line 1: `[VacuumWorld.jpeg](attachment:VacuumWorld.jpeg)'


In [516]:
## Vaccum Cleaner Agent Problem Definition

In [517]:
vc_initial_state = ('A', False, False)

def vc_actions(state):
    return ["suck","move_right","move_left"]
    
def vc_result(state, action):
    loc, A_clean, B_clean = state
    match loc:
        case "A":
            match action:
                case "suck":
                    A_clean = True
                case "move_right":
                    loc = "B"
        case "B":
            match action:
                case "suck":
                    B_clean = True
                case "move_left":
                    loc = "A"
    return (loc, A_clean, B_clean)

def vc_goal_test(state):
    _, A_clean, B_clean = state
    return A_clean and B_clean

def vc_cost(state, action):
    return 1

VC_problem = Problem(vc_initial_state, vc_actions, vc_result, vc_goal_test, vc_cost)

In [518]:
graph_search(VC_problem)


 Solution:

Initial State: ('A', False, False)
Action: suck   Result: ('A', True, False)   Cost: 1
Action: move_right   Result: ('B', True, False)   Cost: 2
Action: suck   Result: ('B', True, True)   Cost: 3
Goal Achieved


In [519]:
graph_search(VC_problem, "DFS")


 Solution:

Initial State: ('A', False, False)
Action: move_right   Result: ('B', False, False)   Cost: 1
Action: suck   Result: ('B', False, True)   Cost: 2
Action: move_left   Result: ('A', False, True)   Cost: 3
Action: suck   Result: ('A', True, True)   Cost: 4
Goal Achieved


In [520]:
graph_search(VC_problem, "UCS")


 Solution:

Initial State: ('A', False, False)
Action: suck   Result: ('A', True, False)   Cost: 1
Action: move_right   Result: ('B', True, False)   Cost: 2
Action: suck   Result: ('B', True, True)   Cost: 3
Goal Achieved
